# Group Chat Orchestration Pattern with Semantic Kernel

This notebook demonstrates the Group Chat Orchestration pattern using Microsoft Semantic Kernel in Python. Group chat orchestration involves multiple agents collaborating simultaneously on a shared task, leveraging Azure OpenAI services for intelligent responses.

### Key Features:
- **Collaborative Execution**: Agents work together in real-time to address a shared task.
- **Task Specialization**: Each agent contributes based on its expertise.
- **Dynamic Interaction**: Agents can respond to each other's outputs, creating a dynamic and interactive workflow.

### Example:
**Scientific Discussion**: Multiple agents (PhysicsExpert, ChemistryExpert, and BiologyExpert) collaborate to answer the question "What is the impact of temperature on living organisms?".

The notebook utilizes `GroupChatOrchestration` and Semantic Kernel's agent orchestration framework to manage and execute tasks collaboratively, showcasing the power and flexibility of Semantic Kernel.

In [ ]:
# Import required libraries
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.agents.orchestration.group_chat import GroupChatOrchestration
from semantic_kernel.agents.runtime import InProcessRuntime

# Azure OpenAI configuration
api_key = "YOUR_AZURE_OPENAI_API_KEY"
endpoint = "https://YOUR_RESOURCE_NAME.openai.azure.com/"
deployment_name = "YOUR_DEPLOYMENT_NAME"

# Initialize Azure OpenAI service
openai_service = AzureChatCompletion(api_key=api_key, endpoint=endpoint, deployment_name=deployment_name)

**Expected Output**: Package installation messages, successful imports, and Azure OpenAI service configuration confirmation.

In [2]:
# Define agents with descriptions
physics_agent = ChatCompletionAgent(
    name="PhysicsExpert",
    description="An expert in physics, focusing on the impact of temperature on physical systems.",
    instructions="You are a physics expert. Explain the impact of temperature on physical systems, such as the behavior of gases and solids.",
    service=openai_service,
)

chemistry_agent = ChatCompletionAgent(
    name="ChemistryExpert",
    description="An expert in chemistry, specializing in temperature effects on chemical reactions.",
    instructions="You are a chemistry expert. Discuss how temperature affects chemical reactions and molecular stability.",
    service=openai_service,
)

biology_agent = ChatCompletionAgent(
    name="BiologyExpert",
    description="An expert in biology, analyzing temperature effects on living organisms.",
    instructions="You are a biology expert. Describe the effects of temperature on living organisms, including metabolism and survival.",
    service=openai_service,
)

agents = [physics_agent, chemistry_agent, biology_agent]

**Expected Output**: Confirmation messages showing that the scientific expert agents (Physics, Chemistry, Biology) have been created and the group chat orchestration is configured.

In [ ]:
# Set up group chat orchestration
from semantic_kernel.agents import RoundRobinGroupChatManager
import asyncio
import traceback

runtime = InProcessRuntime()
manager = RoundRobinGroupChatManager(max_rounds=5)  # Use a round-robin manager for the group chat
# Ensure all agents have descriptions
for agent in agents:
    if not agent.description:
        raise ValueError(f"Agent {agent.name} must have a description.")

group_chat_orchestration = GroupChatOrchestration(members=agents, manager=manager)

# Start runtime
runtime.start()

# Execute group chat orchestration
try:
    print("Starting group chat orchestration...")
    orchestration_result = await group_chat_orchestration.invoke(
        task="What is the impact of temperature on living organisms?",
        runtime=runtime,
    )

    # Collect results
    print("Waiting for orchestration result...")
    value = await orchestration_result.get(timeout=300)  # Increased timeout to 300 seconds
    print("***** Group Chat Results *****")
    for agent_result in value:
        try:
            if isinstance(agent_result, tuple):
                name, content = agent_result  # Unpack tuple
                print(f"{name}: {content}")
            else:
                print(f"Unknown Agent: {agent_result}")
        except Exception as e:
            print(f"Error processing agent result: {e}")
except asyncio.TimeoutError:
    print("The orchestration timed out. Consider increasing the timeout or debugging agent execution.")
except Exception as e:
    print("An error occurred during orchestration:")
    print(traceback.format_exc())

# Stop runtime
await runtime.stop_when_idle()

**Expected Output**: A multi-turn conversation between the expert agents discussing the impact of temperature on living organisms. You'll see each agent contributing their expertise in a collaborative discussion format, with agents building on each other's insights.